<a href="https://colab.research.google.com/github/Anushajadav/Data-pipeline-using-python/blob/main/data_pipline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as nm  

import matplotlib.pyplot as mtp  

import pandas as pd  

# 1 Take a pandas dataframe
dataset=pd.read_csv('dataset.csv')
preprocessed_dataset=data_pipline(dataset)


In [ ]:
def missing_values(dataset):
    features_nan=[feature for feature in dataset.columns if dataset[feature].isnull().sum()>1 and dataset[feature].dtypes=='O']
    #replace the categorical values with label Missing
    def replace_cat_feature(dataset,features_nan):
        data=dataset.copy()
        data[features_nan]=data[features_nan].fillna('Missing')
        return data

    dataset=replace_cat_feature(dataset,features_nan)
    #missing of numerical
    numerical_with_nan=[feature for feature in dataset.columns if dataset[feature].isnull().sum()>1 and dataset[feature].dtypes!='O']
    for feature in numerical_with_nan:
        ## We will replace by using median since there are outliers
        median_value=dataset[feature].median()
        dataset[feature].fillna(median_value,inplace=True)
    categorical_features=[feature for feature in dataset.columns if dataset[feature].dtype=='O']
    #rare values- 
    for feature in categorical_features:
        temp=dataset.groupby(feature)['target'].count()/len(dataset)
        temp_df=temp[temp>0.01].index
        dataset[feature]=np.where(dataset[feature].isin(temp_df),dataset[feature],'Rare_var')
    return dataset


In [ ]:
#Binary classifcation
def over_under_sampling(data):
     
    class_count_0, class_count_1 = data['target'].value_counts()
    print('Before sampling')
    print(data['target'].value_counts())
    
    # check the ration
    if class_count_0>class_count_1:
        val=class_count_0/class_count_1
    else:
        val=class_count_1/class_count_0
        
    #checking the data is balanced or not
    if val<1.5: # less than 1.5 data is balanced
        print("The data is balanced")
        pass
    else:
        print("data is imbalaced, Sampling required")
        class_count_0, class_count_1 = data['target'].value_counts()
        class_0 = data[data['target'] == 0]
        class_1 = data[data['target'] == 1]
        #checking undersampling or oversampling
        # we have assumed million is large dataset
        if data.shape[0] > 1000000:
            #if large data -Under sampling.
            print("Under sampling required, because the size of data is more than million ")
            if class_count_0>class_count_1:
                print(" No class belongs1 0 - {} > No class belongs 1 - {}".format(class_count_0 ,class_count_1 ))
                print("So we have to take some sample from class 0 with the size equal to class 1 ")
                class_0_under = class_0.sample(class_count_1)
                test_under = pd.concat([class_0_under, class_1], axis=0)
                print("After sampling")
                print(test_under['target'].value_counts())
                return test_under
            else:
                print(" No class belongs1 1 - {} > No class belongs 0 - {}".format(class_count_1 ,class_count_0 ))
                print("So we have to take some sample from class 1 with the size equal to class 0 ")
                class_1_under = class_1.sample(class_count_0)
                test_under = pd.concat([class_0_under, class_0], axis=0)
                print("After sampling")
                print(test_under['target'].value_counts())
                return test_under

        else:
            #for  small-medium data we do -over sampling.
            print("over sampling required")
            if class_count_0>class_count_1:
                print(" No class belongs1 0 - {} > No class belongs 1 - {}".format(class_count_0 ,class_count_1 ))
                print("So we have to add more sample to class 1")
                class_oversampling_1 = class_1.sample(class_count_0, replace=True)
                data_oversampling = pd.concat([class_oversampling_1, class_0], axis=0)
                print("After sampling")
                print(data_oversampling['target'].value_counts())
                return data_oversampling
            else:
                print(" No class belongs 1 - {} > No class belongs 0 - {}".format(class_count_1 ,class_count_0 ))
                print("So we have to add more sample to class 0")
                class_oversampling_0 = class_0.sample(class_count_1, replace=True)
                data_oversampling = pd.concat([class_oversampling_0, class_0], axis=0)
                print("After sampling")
                print(data_oversampling['target'].value_counts())
                return data_oversampling

In [ ]:
def correlation_matrix(dataset):
    import seaborn as sns
    #Using Pearson Correlation
    plt.figure(figsize=(20,20))
    cor = dataset.corr()
    sns.heatmap(cor, annot=True, cmap=plt.cm.CMRmap_r)
    plt.show()

    def correlation(dataset, threshold):
        col_corr = set()  # Set of all the names of correlated columns
        corr_matrix = dataset.corr()
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                    colname = corr_matrix.columns[i]  # getting the name of column
                    col_corr.add(colname)
        return col_corr

    columns_correlated=correlation(dataset,90)
    print("highly correlated columns",columns_correlated)
    dataset.drop(columns_correlated,axis=1)
    return dataset

In [ ]:
#Inferential Statistics - wl take some data for testing ,based on efficiency will apply to rest data.
#   example - Vaccine. First we take small sample  from population for checking vaccine effieciency and then infer the vaccine for whole population
#descriptive Statistics- We conduct on entire population. 




How can we remove outlier.
USING Percentile:
50 percentile-means 50% data points are less than 50% percentile.
100 percentile: for which data point the values are less than or equal to that value.
How to find the percentile?

say 25 percntile.
1) Find the lenght of column.
2) 25% of (length of column)

Mode: most frequency used.

IQR.
25th percentile .
lets consider we hv 20 samples.
25% of 20 is 25/100 *20 =5  q1
we shd find the value bw 5th and 6th element.
 
75% of 20  = 75/100*20= 15   q3
we shd find the record bw 15 and 16th row.

25TH percentile Q1
50TH Q2
75TH Q3


IQR= Q3-Q1.

to remove outliers...we hv to come with lower limit and upper limit.

lower limit= Q1-1.5*IQR
upper limit=Q3-1.5*IQR

WE can exclude the lower_limit and upper_limit.

 







In [ ]:
def descriptive_statics(dataset):
      report={}
      numerical_feature =[feature for feature in dataset.columns if dataset[feature].dtypes!='O']
      
      report_des={feature:[] for feature in numerical_feature}
      datasets=dataset.copy()
      datasets.fillna(0)
      for feature in numerical_feature:
        
            #count
            count=len(datasets[feature])
            report_des[feature].append(count)
            
            #mean
            mean= mean=np.round(sum(datasets[feature])/count,6)
            report_des[feature].append(mean)

            #find the median
             
            sorted_data=sorted(datasets[feature]) 
            median=0
            if count%2==0:
                 median_=(sorted_data[int(count/2)]+sorted_data[int(count/2)-1])/2
           
            else:
                 median_=sorted_data[count/2]
            report_des[feature].append(median_)        
             

            #mode.
            mode_= dict(dataset[feature].value_counts()) 
            mode_ = sorted(mode_.items(), key=lambda x: x[1], reverse=True)
            mode_[0][0]
            report_des[feature].append(mode_[0][0])

            #min
            min_=sorted(datasets[feature])[0]
            report_des[feature].append(min_)

            #max
            max_=sorted(datasets[feature],reverse=True)[0]
            report_des[feature].append(max_)
            
            #25%            

            index=0.25*count
             
            if index%10==0:   
                index=int(index)
                percentile_25th=sorted_data[index]
            else:
                index=int(index)
                percentile_25th=(sorted_data[index]+sorted_data[index+1])/2
            report_des[feature].append(percentile_25th)

            #75%
            index=0.75*count
             
            if index%10==0:
                index=int(index)
                percentile_75th=sorted_data[index]
            else:
                index=int(index)
                percentile_75th=(sorted_data[index]+sorted_data[index+1])/2

            report_des[feature].append(percentile_75th) 

            #IQR
            IQR=percentile_75th-percentile_25th
            report_des[feature].append(IQR) 
      reports=pd.DataFrame(report_des)
      reports.index=['count','mean','medean','mode','min','max','25%','75%','IQR']
      return reports

In [ ]:
#assuming the dependent variable as target and primary key as id. assuming for numerical values has outlier.
def data_pipine(data):
    data.columns = [*data.columns[:-1], 'target']
    #2 prints the null values present in each column
    features_has_na=[features for features in data.columns if data[features].isnull().sum()>1]
    for feature in features_has_na:
        print(feature, np.round(data[feature].isnull().mean(), 4),  ' % missing values')

    #2 Missing value handeling
    data=missing_values(data)
    
    #4 categorical transformation to int.
    categorical_features=[feature for feature in data.columns if dataset[feature].dtype=='O']
    for feature in categorical_features:
        labels_ordered=data.groupby([feature])['target'].mean().sort_values().index
        labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
        data[feature]=data[feature].map(labels_ordered)
    
    #convert all the values bw 0 and 1
    feature_scaling=[feature for feature in data.columns if feature not in ['Id','target']]

    from sklearn.preprocessing import MinMaxScaler
    scaler=MinMaxScaler()
    scaler.fit(data[feature_scaling])
    data = pd.concat([data[['Id', 'target']].reset_index(drop=True),
                      pd.DataFrame(scaler.transform(data[feature_scaling]), columns=feature_scaling)],
                      axis=1)
    
    # 3) Considering sampling ration as 60:40 for only binary classification
    if len(data['target'].value_counts())==2:
      data=over_under_sampling(data)
    
    #5 correlation matrix 
    data=correlation_matrix(data)
    #generate descriptive  statistics report 

    reports=descriptive_statics(data) 
    print(reports)